Data Tidying


In [12]:
import pandas as pd

df = pd.read_csv('data/player_stats_injuries_clean.csv')

df.head()


,PLAYER_ID,PLAYER_NAME,SEASON,SEASON_NUM,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,GP,MIN,USG_PCT,...,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,TEAM,INJURED ON,RETURNED,DAYS MISSED,INJURED_TYPE
0,1630639,A.J. Lawson,22-23,22.5,22.0,78,179,13,4.8,0.182,...,1.39,0.62,0.0,0.0,0.2,Knicks,2022-04-10,2020-07-31,4.0,Sprained_ankle
1,1631260,AJ Green,22-23,22.5,23.0,77,190,32,9.4,0.160,...,2.61,1.84,0.1,0.0,0.3,Knicks,2022-04-10,2020-07-31,4.0,Sprained_ankle
2,1631100,AJ Griffin,22-23,22.5,19.0,78,220,66,19.4,0.174,...,2.08,1.49,0.4,0.0,0.7,Knicks,2022-04-10,2020-07-31,4.0,Sprained_ankle
3,203932,Aaron Gordon,22-23,22.5,27.0,80,235,61,30.1,0.206,...,2.94,2.11,1.8,2.1,6.8,Nuggets,2023-02-02,2023-02-07,5.0,Sprained_ankle
4,1628988,Aaron Holiday,22-23,22.5,26.0,72,185,57,13.7,0.128,...,4.17,3.82,0.1,0.0,0.4,Knicks,2022-04-10,2020-07-31,4.0,Sprained_ankle


Check for Duplicates (attention: in our data set possible to have the same person but a different injury or the same person having the same injury a second time)

In [13]:
duplicates = df[df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [PLAYER_ID, PLAYER_NAME, SEASON, SEASON_NUM, AGE, PLAYER_HEIGHT_INCHES, PLAYER_WEIGHT, GP, MIN, USG_PCT, PACE, POSS, FGA_PG, DRIVES, DRIVE_FGA, DRIVE_PASSES, DIST_MILES, AVG_SPEED, PULL_UP_FGA, PULL_UP_FG3A, TOUCHES, FRONT_CT_TOUCHES, AVG_SEC_PER_TOUCH, AVG_DRIB_PER_TOUCH, ELBOW_TOUCHES, POST_TOUCHES, PAINT_TOUCHES, TEAM, INJURED ON, RETURNED, DAYS MISSED, INJURED_TYPE]
Index: []

[0 rows x 32 columns]


No ducplicates were found.
But, we noticed that there is data for the same players but from different seasons. 
-> different/same injuries in different seasons but for the same player

Handle missing Values (figure out what to do with them!)


In [ ]:
#Missing values in columns
missing_values = df.isna().sum()
print(missing_values)

#Missing values in the whole dataset
total_missing_values = df.isnull().sum().sum()
print('total:',total_missing_values)

#Missing values in rows
missing_values_per_row = df.isnull().sum(axis=1)
print('missing:',missing_values_per_row)


missing: 0       0
1       0
2       0
3       0
4       0
       ..
5573    0
5574    0
5575    0
5576    0
5577    0
Length: 5578, dtype: int64


Create the Injury_Severity Column based on missing days

In [22]:

# Define the function to categorize injury severity
def categorize_injury_severity(days_missed):
    if days_missed <= 3:
        return 'Mild'
    elif days_missed <= 7:
        return 'Moderate'
    elif days_missed <= 14:
        return 'Severe'
    else:
        return 'Grave'

# Apply the function to create the Injury_Severity column
df['Injury_Severity'] = df['DAYS MISSED'].apply(categorize_injury_severity)

# Display the first few rows of the updated dataset
print(df.head())

# Save the updated dataset
df.to_csv('updated_player_stats_injuries.csv', index=False)


   PLAYER_ID    PLAYER_NAME SEASON  SEASON_NUM   AGE PLAYER_HEIGHT_INCHES  \
0    1630639    A.J. Lawson  22-23        22.5  22.0                   78   
1    1631260       AJ Green  22-23        22.5  23.0                   77   
2    1631100     AJ Griffin  22-23        22.5  19.0                   78   
3     203932   Aaron Gordon  22-23        22.5  27.0                   80   
4    1628988  Aaron Holiday  22-23        22.5  26.0                   72   

  PLAYER_WEIGHT  GP   MIN  USG_PCT  ...  AVG_DRIB_PER_TOUCH  ELBOW_TOUCHES  \
0           179  13   4.8    0.182  ...                0.62            0.0   
1           190  32   9.4    0.160  ...                1.84            0.1   
2           220  66  19.4    0.174  ...                1.49            0.4   
3           235  61  30.1    0.206  ...                2.11            1.8   
4           185  57  13.7    0.128  ...                3.82            0.1   

   POST_TOUCHES  PAINT_TOUCHES     TEAM  INJURED ON    RETURNED DAYS